In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - 2-spheroid_disk_decomp.ipynb
# AUTHOR - James Lane
# PROJECT - tng-dfs
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Make a figure showing the spheroid-disk decomposition.
'''

__author__ = "James Lane"

In [ ]:
### Imports

# Basic
import numpy as np
import sys, os, pdb, copy, glob, subprocess, dill as pickle, h5py

# Matplotlib
import matplotlib as mpl
from matplotlib import pyplot as plt

# Astropy
from astropy import units as apu

# Project-specific
sys.path.insert(0,'../../src/')
from tng_dfs import cutout as pcutout
from tng_dfs import io as pio
from tng_dfs import kinematics as pkin
from tng_dfs import plot as pplot
from tng_dfs import tree as ptree
from tng_dfs import util as putil

### Notebook setup

%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [ ]:
# %load ../../src/nb_modules/nb_setup.txt
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','MW_ANALOG_DIR','RO','VO','ZO','LITTLE_H',
            'MW_MASS_RANGE']
data_dir,mw_analog_dir,ro,vo,zo,h,mw_mass_range = \
    putil.parse_config_dict(cdict,keywords)

# MW Analog 
mwsubs,mwsubs_vars = putil.prepare_mwsubs(mw_analog_dir,h=h,
    mw_mass_range=mw_mass_range,return_vars=True,force_mwsubs=False,
    bulge_disk_fraction_cuts=False)

# Figure path
fig_dir = './fig/'
epsen_fig_dir = '/epsen_data/scr/lane/projects/tng-dfs/figs/notebooks/paper/'
os.makedirs(fig_dir,exist_ok=True)
os.makedirs(epsen_fig_dir,exist_ok=True)
show_plots = False

# Load tree data
# tree_primary_filename = os.path.join(mw_analog_dir,
#     'major_mergers/tree_primaries.pkl')
# with open(tree_primary_filename,'rb') as handle: 
#     tree_primaries = pickle.load(handle)
# tree_major_mergers_filename = os.path.join(mw_analog_dir,
#     'major_mergers/tree_major_mergers.pkl')
# with open(tree_major_mergers_filename,'rb') as handle:
#     tree_major_mergers = pickle.load(handle)
n_mw = len(mwsubs)

### Plotting Information

In [ ]:
# Column-sized figure width
columnwidth = 244./72.27 # In inches, from pt
# Full-sized figure width
textwidth = 508./72.27 # In inches, from pt

### Gather circularity and kinematic data

In [ ]:
supp_dir = data_dir+'supplementary_data/'
stellar_circs_url = mwsubs_vars['sim']['files']['stellar_circs']
stellar_circs_filename = supp_dir+stellar_circs_url.split('/')[-1]
if not os.path.isfile(stellar_circs_filename):
    putil.get(stellar_circs_url,directory=supp_dir)
else:
    print('Stellar circs file already downloaded.')

fsc = h5py.File(stellar_circs_filename,'r')
snap_key = 'Snapshot_99'
# fsc['Snapshot_99'].keys()
# _keys_load = ['SubfindID',
#               'CircAbove07Frac',
#               'CircAbove07Frac_allstars',
#               'CircAbove07MinusBelowNeg07Frac',
#               'CircAbove07MinusBelowNeg07Frac_allstars',
#               'CircTwiceBelow0Frac',
#               'CircTwiceBelow0Frac_allstars',]
_keys_load = list(fsc[snap_key].keys())
scdata = {}
for key in _keys_load:
    scdata[key] = fsc[snap_key][key][()]
scmask = np.isin(scdata['SubfindID'],mwsubs['id'])
assert np.all( scdata['SubfindID'][scmask] == mwsubs['id'] ) # Make sure actual masking isn't required

### Get Kappa

In [ ]:
verbose = True
kappas = np.zeros(n_mw)

for i in range(n_mw):
    # if i > 2: continue
    if verbose: print(f'Getting MW {i+1}/{n_mw}', end='\r')

    # Get the primary
    snapnum = mwsubs[i]['snap']
    z0_sid = mwsubs[i]['id']
    primary_filename = putil.get_cutout_filename(mw_analog_dir,
        snapnum, z0_sid)
    co = pcutout.TNGCutout(primary_filename)
    co.center_and_rectify()

    orbs = co.get_orbs('stars')
    r = orbs.r().value
    rhm = co.get_half_mass_radius('stars').value
    mask = (r < 2*rhm)
    masses = co.get_masses('stars', physical=True).value
    vR, vT, vz = orbs.vR().value, orbs.vT().value, orbs.vz().value
    vmag = np.sqrt(vR**2 + vT**2 + vz**2)

    kappas[i] = np.sum(masses[mask] * vT[mask]**2)/\
        np.sum(masses[mask] * vmag[mask]**2)

### Make the figure, showing the proxies as well as kappa

In [ ]:
non_p23_mw_analog_ids = [394621, 480802]

# Some parameters
xkey = 'CircTwiceBelow0Frac'
ykey = 'CircAbove07MinusBelowNeg07Frac'
xlim = [-0.1,1.1]
ylim = [-0.1,1.1]
nbin = 10
# fig_title = 'Within twice stellar half-mass radius'
bulge_cut = 0.45
disk_cut = 0.35
ec = 'Black'
s = 36

kappa_cmap = mpl.colormaps.get_cmap('plasma')
kappa_vmin = 0.2
kappa_vmax = 0.8

# Make some plots of properties from this file
fig = plt.figure()
gs = mpl.gridspec.GridSpec(4,4)
ax = plt.subplot(gs[1:,:3])
axt = plt.subplot(gs[0,:3])
axr = plt.subplot(gs[1:,3])

sc_bulge_disk_mask = (scdata[xkey][scmask] < bulge_cut) & \
                        (scdata[ykey][scmask] > disk_cut)

# for j in range(np.sum(sc_bulge_disk_mask)):
#     if mwsubs['id'][sc_bulge_disk_mask][j] in non_p23_mw_analog_ids:
#         mk, zorder = 'o', 2
#     else:
#         mk, zorder = 'o', 1
pts1 = ax.scatter( scdata[xkey][scmask][sc_bulge_disk_mask],
                scdata[ykey][scmask][sc_bulge_disk_mask], s=s,
                marker='o', c=kappas[sc_bulge_disk_mask], edgecolor=ec, 
                zorder=1, vmin=kappa_vmin, vmax=kappa_vmax, cmap=kappa_cmap)
pts2 = ax.scatter( scdata[xkey][scmask][~sc_bulge_disk_mask],
                scdata[ykey][scmask][~sc_bulge_disk_mask], s=s,
                marker='s', c=kappas[~sc_bulge_disk_mask], edgecolor=ec,
                zorder=1, vmin=kappa_vmin, vmax=kappa_vmax, cmap=kappa_cmap)

cax = fig.add_axes([0.5,0.6,0.15,0.025])
cbar = fig.colorbar(pts1, cax=cax, orientation='horizontal')
cax.xaxis.set_ticks([0.2,0.4,0.6,0.8])
cax.xaxis.set_ticklabels(['0.2','0.4','0.6','0.8'])
cax.set_xlabel(r'$\kappa$')


ax.set_xlabel(r'$2f(\eta < 0)$')
ax.set_ylabel(r'$f(\eta > 0.7) - f(\eta < -0.7)$')
ax.set_xlim([xlim[0],xlim[1]])
ax.set_ylim([ylim[0],ylim[1]])
ax.axvline(bulge_cut, color='k', linestyle='--', zorder=0)
ax.axhline(disk_cut, color='k', linestyle='--', zorder=0)
# ax.plot([-1,2],[-1,2],color='k',linestyle='--',zorder=0)

# Plot marginal distributions
axt.hist(scdata[xkey][scmask], bins=nbin, range=(0,1), facecolor='none',
    edgecolor='Black', zorder=1, histtype='step')
axt.axvline(bulge_cut, color='k', linestyle='--', zorder=0)
axt.set_xlim([xlim[0],xlim[1]])
axt.set_ylabel('N')
axt.tick_params(axis='x', labelbottom=False)

axr.hist(scdata[ykey][scmask], bins=nbin, range=(0,1), facecolor='none',
    edgecolor='Black', zorder=1, histtype='step', orientation='horizontal')
axr.axhline(disk_cut, color='k', linestyle='--', zorder=0)
axr.set_ylim([ylim[0],ylim[1]])
axr.set_xlabel('N')
axr.tick_params(axis='y', labelleft=False)

# Circle showing the Milky Way
mw_bulge_fraction = 0.3
mw_bulge_fraction_error = 0.05
mw_disk_fraction = 0.7
mw_disk_fraction_error = 0.05
# mw_circle = plt.Circle((mw_bulge_fraction,mw_disk_fraction), 
#     mw_bulge_fraction_error ,zorder=0, 
#     alpha=0.25, edgecolor='none', facecolor='Black')
# ax.add_artist(mw_circle)

# Cross showing the Milky Way
ax.errorbar(mw_bulge_fraction, mw_disk_fraction,
    xerr=mw_bulge_fraction_error, yerr=mw_disk_fraction_error,
    marker='none', color='Black', zorder=2)

fig.show()
fig.savefig(fig_dir+'spheroid_disk_decomposition.pdf',bbox_inches='tight')